In [7]:
# importo lo que necesito
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import semopy
from sklearn.datasets import load_iris
from factor_analyzer import FactorAnalyzer
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import silhouette_score
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets import load_iris
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.preprocessing import StandardScaler
from factor_analyzer.factor_analyzer import calculate_kmo
from sklearn.datasets import load_iris
from factor_analyzer import FactorAnalyzer
import matplotlib.pyplot as plt
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from semopy import Model
import io
import seaborn as sns
from semopy.inspector import inspect
from semopy import gather_statistics
import seaborn as sns
from scipy.stats import shapiro

In [9]:
# Data pandas
input_file = 'data_final.xlsx'
output_file = 'data_file.csv'
data = pd.read_excel(input_file)
data.to_csv(output_file, index=False)
data

,Universidad,Publicaciones,Ratio,SolicitudPatentes,Q1,Q2,Posgrados,IndicadorPos
0,Asociación Civil Universidad de Ciencias y Hum...,413,0.184032,0,15,6,0,0
1,Facultad de Teología Pontificia y Civil de Lima,3,0.000000,0,0,0,2,8
2,Pontificia Universidad Católica del Perú,4202,0.393372,85,1269,543,2,251
3,Universidad Andina del Cusco,37,0.285811,2,7,7,2,39
4,Universidad Antonio Ruiz de Montoya,24,0.164212,0,6,1,1,13
...,...,...,...,...,...,...,...,...
85,Universidad de Lima,316,0.328006,28,65,40,2,22
86,Universidad de Piura,435,0.348296,10,124,44,2,73
87,Universidad de San Martín de Porres,931,0.319550,9,127,98,2,121
88,Universidad del Pacífico,562,0.017086,0,184,97,2,52


In [10]:
# Reemplazar 1 por 0.5 y 2 por 1 en la columna 7 (iloc[:, 7])
data.iloc[:, 6] = data.iloc[:, 6].replace({1: 0.5, 2: 1})
data

,Universidad,Publicaciones,Ratio,SolicitudPatentes,Q1,Q2,Posgrados,IndicadorPos
0,Asociación Civil Universidad de Ciencias y Hum...,413,0.184032,0,15,6,0.0,0
1,Facultad de Teología Pontificia y Civil de Lima,3,0.000000,0,0,0,1.0,8
2,Pontificia Universidad Católica del Perú,4202,0.393372,85,1269,543,1.0,251
3,Universidad Andina del Cusco,37,0.285811,2,7,7,1.0,39
4,Universidad Antonio Ruiz de Montoya,24,0.164212,0,6,1,0.5,13
...,...,...,...,...,...,...,...,...
85,Universidad de Lima,316,0.328006,28,65,40,1.0,22
86,Universidad de Piura,435,0.348296,10,124,44,1.0,73
87,Universidad de San Martín de Porres,931,0.319550,9,127,98,1.0,121
88,Universidad del Pacífico,562,0.017086,0,184,97,1.0,52


In [11]:
data = data.iloc[:, :-1]

In [12]:
data

,Universidad,Publicaciones,Ratio,SolicitudPatentes,Q1,Q2,Posgrados
0,Asociación Civil Universidad de Ciencias y Hum...,413,0.184032,0,15,6,0.0
1,Facultad de Teología Pontificia y Civil de Lima,3,0.000000,0,0,0,1.0
2,Pontificia Universidad Católica del Perú,4202,0.393372,85,1269,543,1.0
3,Universidad Andina del Cusco,37,0.285811,2,7,7,1.0
4,Universidad Antonio Ruiz de Montoya,24,0.164212,0,6,1,0.5
...,...,...,...,...,...,...,...
85,Universidad de Lima,316,0.328006,28,65,40,1.0
86,Universidad de Piura,435,0.348296,10,124,44,1.0
87,Universidad de San Martín de Porres,931,0.319550,9,127,98,1.0
88,Universidad del Pacífico,562,0.017086,0,184,97,1.0


In [13]:

# Asumiendo que 'data' es tu DataFrame de pandas con los datos cargados

# Descripción estadística básica (excluyendo la primera columna que es el nombre de la universidad)
descriptive_stats = data.iloc[:, 1:].describe()

# Convertir estadísticas descriptivas a formato LaTeX
descriptive_stats_latex = descriptive_stats.to_latex()

# Pruebas de normalidad (Test de Shapiro-Wilk) para cada variable, excluyendo la primera columna
normality_tests = pd.DataFrame(columns=['Variable', 'Statistic', 'p-value'])
for col in data.columns[1:]:
    stat, p = shapiro(data[col])
    normality_tests = normality_tests.append({'Variable': col, 'Statistic': stat, 'p-value': p}, ignore_index=True)

# Configuración de los gráficos
sns.set(style="whitegrid")
palette = ["navy", "maroon"]

# Guardar gráficos de distribución para cada variable en alta resolución
i = 1  # Inicializar el contador para nombrar las figuras como X_i
for col in data.columns[1:]:
    plt.figure(figsize=(10, 6))
    sns.histplot(data[col], color=palette[(data.columns.get_loc(col) - 1) % len(palette)], kde=True)
    plt.title(f'Distribution of $X_{i}$')  # Nombrar la figura como X_i
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.savefig(f'distribution_X_{i}.png', format='png', dpi=300)  # Guardar en alta resolución
    plt.close()
    i += 1  # Incrementar el contador para el próximo gráfico

# Imprimir las estadísticas descriptivas en formato LaTeX
print(descriptive_stats_latex)


C:\Users\marce\AppData\Local\Temp\ipykernel_32752\2402011083.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  descriptive_stats_latex = descriptive_stats.to_latex()
C:\Users\marce\AppData\Local\Temp\ipykernel_32752\2402011083.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  normality_tests = normality_tests.append({'Variable': col, 'Statistic': stat, 'p-value': p}, ignore_index=True)
C:\Users\marce\AppData\Local\Temp\ipykernel_32752\2402011083.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  normality_tests = normality_tests.append({'Variable':

\begin{tabular}{lrrrrrr}
\toprule
{} &  Publicaciones &      Ratio &  SolicitudPatentes &           Q1 &          Q2 &  Posgrados \\
\midrule
count &      90.000000 &  90.000000 &          90.000000 &    90.000000 &   90.000000 &  90.000000 \\
mean  &     372.388889 &   0.428851 &           5.988889 &    88.088889 &   45.433333 &   0.633333 \\
std   &    1016.662315 &   0.217739 &          18.314662 &   305.615410 &  134.821151 &   0.415973 \\
min   &       0.000000 &   0.000000 &           0.000000 &     0.000000 &    0.000000 &   0.000000 \\
25\%   &      21.000000 &   0.308982 &           0.000000 &     2.000000 &    1.000000 &   0.500000 \\
50\%   &      71.500000 &   0.439022 &           0.000000 &     7.500000 &    5.500000 &   1.000000 \\
75\%   &     291.500000 &   0.555160 &           2.000000 &    37.000000 &   27.250000 &   1.000000 \\
max   &    6389.000000 &   1.000000 &         133.000000 &  2392.000000 &  830.000000 &   1.000000 \\
\bottomrule
\end{tabular}



In [14]:
# Pruebas de normalidad (Test de Shapiro-Wilk) para cada variable, excluyendo la primera columna
normality_tests = pd.DataFrame(columns=['Variable', 'Statistic', 'p-value'])
for col in data.columns[1:]:
    stat, p = shapiro(data[col])
    normality_tests = normality_tests.append({'Variable': col, 'Statistic': stat, 'p-value': p}, ignore_index=True)

# Convertir los resultados de las pruebas de normalidad a formato LaTeX
normality_tests_latex = normality_tests.to_latex(index=False)

# Imprimir los resultados en formato LaTeX
print(normality_tests_latex)

\begin{tabular}{lrr}
\toprule
         Variable &  Statistic &      p-value \\
\midrule
    Publicaciones &   0.367097 & 7.179654e-18 \\
            Ratio &   0.979520 & 1.672049e-01 \\
SolicitudPatentes &   0.365297 & 6.800968e-18 \\
               Q1 &   0.294902 & 8.898616e-19 \\
               Q2 &   0.350255 & 4.343951e-18 \\
        Posgrados &   0.742006 & 2.819779e-11 \\
\bottomrule
\end{tabular}



C:\Users\marce\AppData\Local\Temp\ipykernel_32752\1738337197.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  normality_tests = normality_tests.append({'Variable': col, 'Statistic': stat, 'p-value': p}, ignore_index=True)
C:\Users\marce\AppData\Local\Temp\ipykernel_32752\1738337197.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  normality_tests = normality_tests.append({'Variable': col, 'Statistic': stat, 'p-value': p}, ignore_index=True)
C:\Users\marce\AppData\Local\Temp\ipykernel_32752\1738337197.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  normality_tests = normality_tests.append({'Variable': col, 'Statistic': stat, 'p-value': p}, ignore_index=True)
C:\Users\marce\AppData\Local\Temp\ipykernel_32752\17383371